In [6]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [7]:
!pip install langchain langchain-community

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 KB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.3/198.3 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.1/326.1 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.6/241.6 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.9/222.9 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 KB 13.0 MB/s eta 0:00:00


In [8]:
!pip install langgraph==0.3 

Defaulting to user installation because normal site-packages is not writeable


In [9]:
# ✅ LLM 초기화
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ✅ 스타일 체크 노드
style_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 코드 스타일 전문가입니다. 코드 스타일 문제를 찾아 리스트로 알려주세요."),
    ("human", "{code}")
])
style_chain = style_prompt | llm

def style_check_node(state):
    result = style_chain.invoke({"code": state["code"]})
    state["style_issues"] = result.content.split("\n")
    return state

# ✅ 버그 체크 노드
bug_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 코드 리뷰 시스템입니다. 버그 가능성이 있는 부분을 찾아 설명해 주세요."),
    ("human", "{code}")
])
bug_chain = bug_prompt | llm

def bug_check_node(state):
    result = bug_chain.invoke({"code": state["code"]})
    state["bug_issues"] = result.content.split("\n")
    return state

# ✅ 최적화 체크 노드
opt_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 성능 최적화 전문가입니다. 개선 가능한 성능 문제와 그 이유를 알려주세요."),
    ("human", "{code}")
])
opt_chain = opt_prompt | llm

def optimize_check_node(state):
    result = opt_chain.invoke({"code": state["code"]})
    state["optimize_suggestions"] = result.content.split("\n")
    return state

# ✅ 개선 코드 생성 노드
improve_prompt = ChatPromptTemplate.from_messages([
    ("system", "이전 분석 내용을 참고하여 개선된 코드를 생성해 주세요."),
    ("human", "원래 코드:\n{code}\n\n스타일 문제:\n{style}\n\n버그:\n{bugs}\n\n최적화 제안:\n{opt}")
])
improve_chain = improve_prompt | llm

def improve_code_node(state):
    result = improve_chain.invoke({
        "code": state["code"],
        "style": "\n".join(state.get("style_issues", [])),
        "bugs": "\n".join(state.get("bug_issues", [])),
        "opt": "\n".join(state.get("optimize_suggestions", []))
    })
    state["improved_code"] = result.content
    return state

# ✅ LangGraph 연결
from langgraph.graph import StateGraph
from typing import TypedDict, List

class CodeState(TypedDict, total=False):
    code: str
    style_issues: List[str]
    bug_issues: List[str]
    optimize_suggestions: List[str]
    improved_code: str

graph = StateGraph(state_schema=CodeState)
graph.add_node("style_check", style_check_node)
graph.add_node("bug_check", bug_check_node)
graph.add_node("optimize_check", optimize_check_node)
graph.add_node("code_improve", improve_code_node)

graph.add_edge("style_check", "bug_check")
graph.add_edge("bug_check", "optimize_check")
graph.add_edge("optimize_check", "code_improve")

graph.set_entry_point("style_check")
graph.set_finish_point("code_improve")

app = graph.compile()

# ✅ 실행 예시
input_code = {
    "code": "def add(x,y): return x+y"
}
result = app.invoke(input_code)
print(result["improved_code"])

/tmp/ipykernel_9144/353687369.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


아래는 원래 코드의 개선된 버전입니다. 이 코드는 가독성을 높이고, 타입 검사를 추가하며, 문서화를 통해 사용법을 명확히 하고 있습니다.

```python
def add(x: float, y: float) -> float:
    """두 수를 더하여 반환합니다.

    Args:
        x (float): 첫 번째 숫자.
        y (float): 두 번째 숫자.

    Returns:
        float: x와 y의 합.

    Raises:
        TypeError: x 또는 y가 숫자가 아닐 경우.
        ValueError: x 또는 y가 None일 경우.
    """
    if x is None or y is None:
        raise ValueError("Arguments cannot be None")
    
    if not isinstance(x, (int, float)) or not isinstance(y, (int, float)):
        raise TypeError("Both arguments must be int or float")
    
    return x + y
```

### 개선 사항 설명:

1. **타입 힌트**: 매개변수와 반환값에 대한 타입 힌트를 추가하여 함수의 사용법을 명확히 했습니다.

2. **문서화**: 함수의 목적, 매개변수, 반환값, 그리고 발생할 수 있는 예외에 대한 설명을 추가했습니다. 이는 다른 개발자가 이 함수를 사용할 때 큰 도움이 됩니다.

3. **타입 검사**: `x`와 `y`가 `None`인지 확인하고, 숫자가 아닌 경우에는 `TypeError`를 발생시킵니다. 이를 통해 함수의 안정성을 높였습니다.

4. **예외 처리**: `None` 값에 대한 처리를 추가하여, 잘못된 인수가 전달될 경우 명확한 오류 메시지를 제공합니다.

이러한 개선을 통해 함수는 더 안전하고 사용하기 쉬우며, 다른 개발자들이 이해하기 쉬운 코드가 되었